In [1]:
! pip install rapidocr-onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.0 MB/s eta 0:00:00


In [2]:
! pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.0 MB/s eta 0:00:00


In [3]:
! pip install PyMuPDF sentence-transformers faiss-cpu Pillow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 10.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_

In [4]:
! pip install tabula-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 29.3 MB/s eta 0:00:00


In [5]:
! pip install pillow torchvision

In [6]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import os
import fitz  # PyMuPDF
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import random

# Initialize Sentence Transformer Model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize FAISS
dimension = 384  # Dimension of embeddings from all-MiniLM-L6-v2
index = faiss.IndexFlatL2(dimension)

# Function to extract text and images from a PDF
def extract_content_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    texts = []
    images = []

    for page in doc:
        # Extract text
        texts.append(page.get_text())

        # Extract images
        image_list = page.get_images(full=True)
        for img_index, img in enumerate(image_list):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            images.append(image_bytes)  # Storing image bytes, you could process these further

    doc.close()
    return texts, images

# Function to embed text
def embed_text(texts):
    embeddings = model.encode(texts)
    return embeddings

# Load 20 PDFs and process
folder_path = '/content/drive/Shareddrives/298A/Vector DB/extracted_arxiv_papers'
pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]
selected_files = random.sample(pdf_files, min(20, len(pdf_files)))

for pdf_file in selected_files:
    pdf_path = os.path.join(folder_path, pdf_file)
    texts, images = extract_content_from_pdf(pdf_path)
    if texts:
        text_embeddings = embed_text(texts)
        index.add(np.array(text_embeddings, dtype='float32'))  # Add text embeddings to FAISS index

# Optionally, you could save the FAISS index to disk
faiss.write_index(index, 'text_embeddings.index')

print("Indexed", index.ntotal, "text embeddings.")

Indexed 257 text embeddings.


In [15]:
import faiss
import numpy as np

def load_faiss_index(index_path):
    """ Load a FAISS index from the specified file path. """
    return faiss.read_index(index_path)

def print_faiss_index_contents(index):
    """ Print all vectors stored in a FAISS index. """
    print(f"Total number of vectors in the index: {index.ntotal}")
    if index.ntotal > 0:
        for i in range(index.ntotal):
            vector = np.zeros((index.d,), dtype='float32')  # Corrected to create a 1D array
            index.reconstruct(i, vector)  # Retrieve vector by its index
            print(f"Vector {i}: {vector}")  # Print the vector

# Path to the FAISS index file
index_path = 'text_embeddings.index'

# Load the index
index = load_faiss_index(index_path)

# Print contents of the index
print_faiss_index_contents(index)


Streaming output truncated to the last 5000 lines.
  6.95831478e-02  1.39677408e-03  7.67233148e-02  6.10977896e-02
  5.69548011e-02 -4.09566499e-02  3.53050269e-02 -3.85011584e-02
 -1.09626211e-01  3.07551380e-02 -3.46512012e-02 -7.69721577e-04
 -3.74835730e-02 -4.07076403e-02 -4.59438935e-02 -6.22254871e-02
  7.38523155e-02 -6.11172244e-02  1.61684193e-02  1.49084663e-03
 -5.46306372e-02  2.72499248e-02  6.50121868e-02  7.49438927e-02
  1.13536495e-04 -7.66016468e-02  5.20878285e-02  7.74383843e-02
  1.25429546e-02  5.00459224e-02 -7.79786110e-02  5.00856759e-03]
Vector 205: [ 1.02623925e-02 -9.92411096e-03 -5.65223396e-02 -6.77827001e-02
 -3.38532627e-02 -1.48033015e-02 -2.71109492e-02  8.37308988e-02
  1.04294689e-02  5.25813065e-02  2.63272785e-02 -4.95944470e-02
  5.28045148e-02  2.55415170e-03  5.61943725e-02  3.60797122e-02
  8.23723525e-02 -4.14233282e-03 -5.58020547e-02  4.57204022e-02
  8.85267779e-02 -8.75765085e-02 -5.38998842e-02 -1.71920229e-02
 -8.65791515e-02  5.053225

In [11]:
import os
import fitz  # Alias for PyMuPDF
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import io
from PIL import Image

def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    texts = [page.get_text() for page in document]
    document.close()
    print(f"Extracted text from {pdf_path}.")
    return texts

def extract_images_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    images = []
    for page in document:
        for img_index, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            base_image = document.extract_image(xref)
            images.append(base_image["image"])
    document.close()
    print(f"Extracted {len(images)} images from {pdf_path}.")
    return images

def create_image_features(image_bytes, size=(64, 64)):
    img = Image.open(io.BytesIO(image_bytes))
    img = img.convert('RGB').resize(size)
    return np.array(img).flatten()  # Flatten image data to a 1D array

model_text = SentenceTransformer('all-MiniLM-L6-v2')

dimension_text = 384  # Adjust based on the output of Sentence Transformers
dimension_image = 64 * 64 * 3  # Adjust based on the flattened image features
index_text = faiss.IndexFlatL2(dimension_text)
index_image = faiss.IndexFlatL2(dimension_image)

def main(folder_path):
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]
    selected_files = pdf_files[:20]  # Select the first 20 PDFs

    for pdf_file in selected_files:
        pdf_path = os.path.join(folder_path, pdf_file)
        texts = extract_text_from_pdf(pdf_path)
        images = extract_images_from_pdf(pdf_path)

        # Process text to embeddings
        if texts:
            text_embeddings = model_text.encode(texts)
            index_text.add(np.array(text_embeddings, dtype='float32'))
            print(f"Text embeddings processed for {pdf_file}.")

        # Process images to simple features and add to index
        if images:
            image_features = np.array([create_image_features(image) for image in images], dtype='float32')
            index_image.add(image_features)
            print(f"Image features processed and indexed for {pdf_file}.")

    # Save the FAISS indices
    text_index_path = 'text_index.faiss'
    image_index_path = 'image_index.faiss'
    faiss.write_index(index_text, text_index_path)
    faiss.write_index(index_image, image_index_path)

    print(f"Indexed {index_text.ntotal} text embeddings and {index_image.ntotal} image embeddings.")
    print(f"Text index saved to {text_index_path}.")
    print(f"Image index saved to {image_index_path}.")

if __name__ == "__main__":
    google_drive_folder_path = '/content/drive/Shareddrives/298A/Vector DB/extracted_arxiv_papers'
    main(google_drive_folder_path)

Extracted text from /content/drive/Shareddrives/298A/Vector DB/extracted_arxiv_papers/0704_3515_extracted.pdf.
Extracted 8 images from /content/drive/Shareddrives/298A/Vector DB/extracted_arxiv_papers/0704_3515_extracted.pdf.
Text embeddings processed for 0704_3515_extracted.pdf.
Image features processed and indexed for 0704_3515_extracted.pdf.
Extracted text from /content/drive/Shareddrives/298A/Vector DB/extracted_arxiv_papers/0704_3433_extracted.pdf.
Extracted 3 images from /content/drive/Shareddrives/298A/Vector DB/extracted_arxiv_papers/0704_3433_extracted.pdf.
Text embeddings processed for 0704_3433_extracted.pdf.
Image features processed and indexed for 0704_3433_extracted.pdf.
Extracted text from /content/drive/Shareddrives/298A/Vector DB/extracted_arxiv_papers/0705_0693_extracted.pdf.
Extracted 11 images from /content/drive/Shareddrives/298A/Vector DB/extracted_arxiv_papers/0705_0693_extracted.pdf.
Text embeddings processed for 0705_0693_extracted.pdf.
Image features processed

In [13]:
import faiss
import numpy as np

def load_faiss_index(path):
    """ Load a FAISS index from a given file path. """
    return faiss.read_index(path)

def print_all_vectors_from_faiss_index(index):
    """ Print all vectors stored in a FAISS index. """
    print(f"Total number of vectors in index: {index.ntotal}")
    if index.ntotal > 0:
        for i in range(index.ntotal):
            vector = np.zeros((1, index.d), dtype='float32')
            index.reconstruct(i, vector)  # Retrieve vector by its index
            print(f"Vector {i}: {vector}")

# Load the text and image FAISS indices
text_index_path = 'text_index.faiss'
image_index_path = 'image_index.faiss'

text_index = load_faiss_index(text_index_path)
image_index = load_faiss_index(image_index_path)

# Print all vectors from each index
print("Vectors in Text Index:")
print_all_vectors_from_faiss_index(text_index)

print("Vectors in Image Index:")
print_all_vectors_from_faiss_index(image_index)


Vectors in Text Index:
Total number of vectors in index: 257


AssertionError: 

In [12]:
import faiss
import numpy as np

def load_faiss_index(path):
    """ Load a FAISS index from a given file path. """
    return faiss.read_index(path)

def print_faiss_index_info(index):
    """ Print basic information about a FAISS index, such as the number of vectors and the dimension. """
    print(f"Index contains {index.ntotal} vectors, each with dimension {index.d}")
    if index.ntotal > 0:
        # Print the first vector to check
        print("Example vector (first vector):")
        vector = np.zeros((1, index.d), dtype='float32')
        index.reconstruct(0, vector)  # Get the first vector
        print(vector)

def perform_search(index, query_vector, k=5):
    """ Perform a nearest neighbor search in the given FAISS index. """
    distances, indices = index.search(np.array([query_vector], dtype='float32'), k)
    print("Nearest neighbors:")
    for i, idx in enumerate(indices[0]):
        print(f"Index: {idx}, Distance: {distances[0][i]}")

# Load indices
text_index = load_faiss_index('text_index.faiss')
image_index = load_faiss_index('image_index.faiss')

# Print index information
print("Text Index Information:")
print_faiss_index_info(text_index)
print("Image Index Information:")
print_faiss_index_info(image_index)

# Example search (assuming you have a query vector)
# For text: assuming you have a query sentence and model loaded
query_sentence = "Example query text"
query_vector = model_text.encode([query_sentence])[0]
perform_search(text_index, query_vector)


Text Index Information:
Index contains 257 vectors, each with dimension 384
Example vector (first vector):


AssertionError: 